I use the code from Will Koehrsen. Thanks for Will Koehrsen

Data: Có 7 đa ta khác nhau
1)application_train/application_test:Khoản vay chưa hoàn trả:1, khoản vay chưa hoàn trả:0.
2)bureau:Khoản vay trước đây từ các tổ chức tài chính khác
3)bureau_balance:Dữ liệu hàng tháng về các khoản vay trước đây trong văn phòng
4)previous_application:Dữ liệu các khoản vay trước đây tại Home Credit
5)POS_CASH_BALANCE:dữ liệu hàng hàng về các khoảng vay của khách hàng với Home Credit
6)credit_card_balance:dữ liệu hàng tháng về các thẻ tín dụng mà khách hàng sử dụng với Home Credit
7)installments_payment:lịch sử thanh toán với Home Credit


Import:

In [ ]:
!pip install scikit-misc
!pip install scikit-learn==0.21.3

In [ ]:
import numpy as np
import pandas as pd 

# sklearn preprocessing để xử lý các biên phân loại
from sklearn.preprocessing import LabelEncoder

# File quản lý tập tin
import os

# loại bỏ cảnh báo 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for vẽ mô hình
import matplotlib.pyplot as plt
import seaborn as sns

# Đọc Data

Liệt kê tất cả cá tệp dữ liệu có sẵn

In [ ]:
print(os.listdir("../input/home-credit-default-risk"))

In [ ]:
#training Data
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Có 307511 quan sát và 122 biến

In [ ]:
#TestingData
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

# Exploratory Data Analysis

Phân tích dữ liệu khám phá (EDA) là một quy trình kết thúc mở, trong đó tính toán số liệu thống kê và đưa ra số liệu để tìm xu hướng, sự bất thường, mô hình hoặc mối quan hệ trong dữ liệu

Kiểm tra sự phân bố của "TARGET"

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist();

Có nhiều khoản vay được hoàn trả đúng hạn hơn so với các khoản vay không được hoàn trả

# Kiểm tra các giá trị còn thiếu

In [ ]:
# Hàm tính toán các giá trị còn thiếu theo cột 
def missing_values_table(df):
        # Tổng các giá trị còn thiếu
        mis_val = df.isnull().sum()
        
        # Tỷ lệ phần trăm thiếu
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Lập bản với kết quả
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Đổi tên côt
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sắp sếp bảng theo tỉ lệ phần trăm giảm dần
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        #In thông tin cần thiết
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Trả về dữ liệu với thông tin còn thiếu
        return mis_val_table_ren_columns

In [ ]:
missing_values = missing_values_table(app_train)
missing_values.head(20)

# Column Types

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

Hầu hết các biến phân loại có số lượng mục duy nhất tương đối nhỏ, tìm cách xử lý các biến phân loại này

# Mã hóa Categorical Variables

Label encoding: gán từng loại duy nhất trong một biến phân loại với một số nguyên

In [ ]:
# Tạo một nhãn
le = LabelEncoder()
le_count = 0

# Lặp lại qua các cột
for col in app_train:
    if app_train[col].dtype == 'object':
        # nếu 2 hoặc ít hơn unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

One-hot encoding:tạo một cột mới cho mỗi danh mục duy nhất trong một biến phân loại. Mỗi quan sát nhận được 1 trong cột cho loại tương ứng và 0 trong tất cả các cột mới khác.

In [ ]:
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

# Sắp xếp Training and Testing Data

In [ ]:
train_labels = app_train['TARGET']

# Chỉnh lại dữ liệu train và test chỉ giữ lại các cột có trong ca hai tệp dữ liệu
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Thêm "TARGET"
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)


# Quay lại phân tích dữ liệu thăm dò

Các số trong cột DAYS_BIRTH âm bởi vì chúng được ghi lại liên quan đến đơn xin vay hiện tại. Để xem các số liệu thống kê này theo năm, chúng ta có thể nhân đôi -1 và chia cho số ngày trong một năm

In [ ]:
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

The maximum value (besides being positive) is about 1000 years!

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

tập hợp các khách hàng dị thường và xem liệu họ có xu hướng có tỷ lệ mặc định cao hơn hoặc thấp hơn so với các khách hàng còn lại.

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

Điền vào các giá trị dị thường không phải là một số (np.nan) và sau đó tạo một cột boolean mới cho biết giá trị đó có bất thường hay không.

In [ ]:
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

Tạo cột mới và điền vào cột hiện có với np.nan trong dữ liệu test.

In [ ]:
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

# Correlations

Một cách để thử và hiểu dữ liệu là tìm kiếm mối tương quan giữa các tính năng và mục tiêu. Chúng ta có thể tính hệ số tương quan Pearson giữa mọi biến và mục tiêu bằng phương pháp khung dữ liệu .corr.
Hệ số tương quan không phải là phương pháp tốt nhất để biểu thị "mức độ liên quan" của một tính năng, nhưng nó cho chúng ta ý tưởng về các mối quan hệ có thể có trong dữ liệu.

In [ ]:
correlations = app_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

DAYS_BIRTH là mối tương quan tích cực nhất. (ngoại trừ TARGET vì tương quan của một biến với chính nó luôn là 1!) Nhìn vào tài liệu, DAYS_BIRTH là độ tuổi tính theo ngày của khách hàng tại thời điểm cho vay trong những ngày âm (vì bất kỳ lý do gì!). Mối tương quan là tích cực, nhưng giá trị của tính năng này thực sự là âm, có nghĩa là khi khách hàng già đi, họ ít có khả năng mặc định cho khoản vay của mình (ie the target == 0)

# Ảnh hưởng của tuổi tác với việc hoàn trả

In [ ]:
#Tìm mối tương qua giữa birth và target
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

Khi khách hàng già đi, họ có xu hướng trả nợ đúng hạn thường xuyên hơn
Tạo ra một biểu đồ histogram of the age đặt trục x trong nhiều năm để làm cho nó dễ hiểu hơn một chút.

Tiếp theo chúng ta sẽ tạo một kernel density estimation plot (KDE) được tô màu bằng giá trị của mục tiêu. A kernel density estimate plot shows the distribution of a single variable(nó được tạo bằng cách tính toán một hạt nhân, thường là một Gaussian, tại mỗi điểm dữ liệu và sau đó lấy trung bình tất cả các hạt nhân riêng lẻ để phát triển một độ mịn đường cong)

In [ ]:
plt.figure(figsize = (10, 8))

# KDE plot các khoảng vay trả đúng hạng
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / 365, label = 'target == 0')

# KDE plot các khoảng vay không trả đúng hạng
sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / 365, label = 'target == 1')

# Hiện biểu đồ
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

Target == 1 nghiêng về phía cuối của phạm vi trẻ.
Mặc dù đây không phải là một mối tương quan đáng kể (-0.07 correlation coefficient), biến này có thể sẽ hữu ích trong mô hình học máy vì nó ảnh hưởng đến

Cắt loại tuổi thành từng hộp 5 năm. Sau đó, với mỗi hộp,tính giá trị trung bình của mục tiêu, cho ta biết tỷ lệ các khoản vay không được hoàn trả trong từng loại tuổi.

In [ ]:
# Điền thông tin tuổi vào 1 separate dataframe
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# Bin the age data
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
# Group by the hộp và tính trung bình
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize = (8, 8))

# Vẽ đồ thị các thùng tuổi và mức trung bình của mục tiêu dưới dạng biểu đồ thanh
plt.bar(age_groups.index.astype(str), 100 * age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');


Có một xu hướng rõ ràng: những ứng viên trẻ tuổi có nhiều khả năng không trả được khoản vay! Tỷ lệ không trả nợ là trên 10% cho ba nhóm tuổi trẻ nhất và 5% cho nhóm tuổi già nhất.
Đây là thông tin có thể được ngân hàng sử dụng trực tiếp: bởi vì các khách hàng trẻ tuổi ít có khả năng trả nợ, có lẽ họ nên được cung cấp thêm hướng dẫn hoặc mẹo lập kế hoạch tài chính. Điều này không có nghĩa là ngân hàng nên phân biệt đối xử với khách hàng trẻ tuổi, nhưng sẽ rất thông minh khi thực hiện các biện pháp phòng ngừa để giúp khách hàng trẻ thanh toán đúng hạn.

# Exterior Sources

3 biến có tương quan âm mạnh nhất với mục tiêu là EXT_SOURCE_1, EXT_SOURCE_2 và EXT_SOURCE_3. Theo tài liệu này, các tính năng này thể hiện"normalized score from external data source"

Hiển thị các mối tương quan của các tính năng EXT_SOURCE với mục tiêu và giữa chúng với nhau với nhau.

In [ ]:
# Trích xuất các biến EXT_SOURCE và hiển thị tương quan
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize = (8, 6))

# Sơ đồ tương quan
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

Tất cả ba tính năng EXT_SOURCE đều có tương quan âm với mục tiêu, cho thấy rằng khi giá trị của EXT_SOURCE tăng lên, khách hàng có nhiều khả năng trả nợ hơn. Chúng ta cũng có thể thấy rằng DAYS_BIRTH có mối tương quan tích cực với EXT_SOURCE_1 chỉ ra rằng có thể một trong những yếu tố trong điểm số này là tuổi của khách hàng.

Tiếp theo chúng ta có thể xem xét sự phân phối của từng tính năng này được tô màu bởi giá trị của mục tiêu. Điều này sẽ cho chúng ta hình dung ảnh hưởng của biến này đến mục tiêu.

In [ ]:
plt.figure(figsize = (10, 12))

# lặp qua các nguồn
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    # tạo một subplot mới cho mỗi nguồn
    plt.subplot(3, 1, i + 1)
    # plot Trả nợ
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    # plot các khoản vay không được hoàn trả
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)


EXT_SOURCE_3 hiển thị sự khác biệt lớn nhất giữa các giá trị của mục tiêu. Chúng ta có thể thấy rõ rằng tính năng này có một số mối quan hệ với khả năng người nộp đơn trả nợ.

# Pairs Plot

Chúng ta có thể tạo một cặp biểu đồ của các biến EXT_SOURCE và biến DAYS_BIRTH.
Sử dụng thư viện trực quan hóa đáy biển và hàm PairGrid để tạo ra một Lô ghép với các biểu đồ phân tán trên tam giác trên, biểu đồ trên đường chéo và các biểu đồ mật độ hạt nhân 2D và các hệ số tương quan trên tam giác dưới.

In [ ]:
# sao chép dữ liệu for plotting
plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Thêm tuổi của khách hàng trong năm
plot_data['YEARS_BIRTH'] = age_data['YEARS_BIRTH']

# Giảm giá trị na và giới hạn ở 100000 hàng đầu tiên
plot_data = plot_data.dropna().loc[:100000, :]

# Hàm tính hệ số tương quan giữa hai cột
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Tạo đối tượng cặp
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

Màu đỏ biểu thị các khoản vay không được hoàn trả và màu xanh là các khoản vay được trả. Chúng ta có thể thấy các mối quan hệ khác nhau trong dữ liệu. Dường như có mối quan hệ tuyến tính tích cực vừa phải giữa EXT_SOURCE_1 và DAYS_BIRTH (hoặc tương đương YEARS_BIRTH), cho thấy tính năng này có thể tính đến tuổi của khách hàng.

# Feature Engineering

Feature engineering đề cập đến một quá trình tạo gen và có thể liên quan đến cả việc xây dựng tính năng: thêm các tính năng mới từ dữ liệu hiện có và lựa chọn tính năng: chỉ chọn các tính năng quan trọng nhất hoặc các phương pháp giảm kích thước khác. Có nhiều kỹ thuật chúng ta có thể sử dụng để tạo cả tính năng và chọn tính năng.

**Polynomial Features**

Trong phương pháp này, ta tạo ra các tính năng là sức mạnh của các tính năng hiện có cũng như các điều khoản tương tác giữa các tính năng hiện có. Ví dụ: chúng tôi có thể tạo các biến EXT_SOURCE_1 ^ 2 và EXT_SOURCE_2 ^ 2 và cả các biến như EXT_SOURCE_1 x EXT_SOURCE_2, EXT_SOURCE_1 x EXT_SOURCE_2 ^ 2, EXT_SOURCE_1 ^ 2 x EXT_SOURCE Các tính năng này là sự kết hợp của nhiều biến riêng lẻ được gọi là thuật ngữ tương tác vì chúng nắm bắt được sự tương tác giữa các biến.

In [ ]:
#Tạo một khung dữ liệu mới cho polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# imputer để xử lý các giá trị còn thiếu
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Áp đặt các giá trị còn thiếu
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Tạo 1 đa thức xác định mức độ
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

Điều này tạo ra một số lượng đáng kể các tính năng mới. Để có được tên, chúng ta phải sử dụng các tính năng đa thức get_feature_names

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

Xem liệu bất kỳ tính năng mới nào trong số này có tương quan với mục tiêu hay không.

In [ ]:
# Tạo một khung dữ liệu của các tính năng
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Add in the target
poly_features['TARGET'] = poly_target

# Tìm sự tương quan với target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Hiển thị tính tích cực và tiêu cực
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

Một số biến mới có mối tương quan lớn hơn (về độ lớn tuyệt đối) với target hơn là các tính năng đầu.Thêm các tính năng này vào một bản sao của dữ liệu đào tạo và thử nghiệm và sau đó đánh giá các mô hình có và không có các tính năng

In [ ]:
# Đặt tính năng kiểm tra vào khung dữ liệu
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Hợp nhất các tính năng đa thức vào khung dữ liệu đào tạo
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Hợp nhất các tính năng đa ngôn ngữ vào khung dữ liệu thử nghiệm
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

# Domain Knowledge Features

Tạo ra một vài tính năng cố gắng nắm bắt những gì có thể quan trọng để cho biết khách hàng có mặc định cho khoản vay hay không
CREDIT_INCOME_PERCENT: tỷ lệ phần trăm của số tiền tín dụng so với thu nhập của khách hàng
ANNUITY_INCOME_PERCENT: tỷ lệ phần trăm của tiền hằng năm cho vay so với thu nhập của khách hàng
CREDIT_TERM: thời hạn thanh toán tính theo tháng
DAYS_EMPLOYED_PERCENT: tỷ lệ phần trăm số ngày làm việc so với tuổi của khách hàng

In [ ]:
app_train_domain = app_train.copy()
app_test_domain = app_test.copy()

app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

Make the same KDE plot colored by the value of the TARGET.

In [ ]:
plt.figure(figsize = (12, 20))
# Duyệt qua các tính năng mới
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    #tạo một subplot mới cho mỗi nguồn
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

# Đường cơ sở

Đối với đường cơ sở, chúng ta có thể đoán cùng một giá trị cho tất cả các ví dụ trên tập kiểm tra
Đoán 0,5 cho tất cả các quan sát trên bộ thử nghiệm
Sử dụng hồi quy logistic để dự đoàn đường cơ sở

# Thực hiện hồi quy


Để có được đường cơ sở, ta sẽ sử dụng tất cả các tính năng sau khi mã hóa các biến phân loại
Xử lý trước dữ liệu bằng cách điền vào các giá trị còn thiếu (chỉ định) và chuẩn hóa phạm vi của các tính năng (tính năng chia tỷ lệ)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Lấy mục tiêu từ dữ liệu huấn luyện
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Trung bình các giá trị thiếu
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Chuyển đổ dữ liệu train và test
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Sử dụng cú pháp mô hình Scikit-Learn quen thuộc: trước tiên tạo mô hình, sau đó chúng tôi huấn luyện mô hình bằng cách sử dụng .fit và sau đó đưa ra dự đoán về dữ liệu thử nghiệm bằng cách sử dụng .predict_proba

In [ ]:
from sklearn.linear_model import LogisticRegression

# Tạo mô hình với tham số chính quy đã chỉ định
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

Sử dụng phương pháp dự đoán mô hình. Điều này trả về một mảng m x 2 trong đó m là số lượng quan sát. Cột đầu tiên là xác suất của mục tiêu là 0 và cột thứ hai là xác suất của mục tiêu là 1. Do muốn xác nhận khoảng vay không hoàng trả nên chọn cột 2.

In [ ]:
log_reg_pred = log_reg.predict_proba(test)[:, 1]

Các dự đoán phải ở định dạng được hiển thị trong tệp sample_submission.csv, trong đó chỉ có hai cột: SK_ID_CURR và TARGET

In [ ]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

Các dự đoán đại diện cho xác suất từ 0 đến 1 rằng khoản vay sẽ không được hoàn trả.

In [ ]:
# Save the submission to a csv file
submit.to_csv('log_reg_baseline.csv', index = False)

Đường cơ sở hồi quy logistic sẽ đạt khoảng 0,671 khi được gửi.

# Cải thiện Model bằng Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# Train on the training data
random_forest.fit(train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline.csv', index = False)

Mô hình này sẽ đạt khoảng 0,678 khi gửi.

# Dự đoán bằng cách sử dụng các tính năng thiết kế

In [ ]:
poly_features_names = list(app_train_poly.columns)

# Impute the polynomial features
imputer = Imputer(strategy = 'median')

poly_features = imputer.fit_transform(app_train_poly)
poly_features_test = imputer.transform(app_test_poly)

# Scale the polynomial features
scaler = MinMaxScaler(feature_range = (0, 1))

poly_features = scaler.fit_transform(poly_features)
poly_features_test = scaler.transform(poly_features_test)

random_forest_poly = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# Train on the training data
random_forest_poly.fit(poly_features, train_labels)

# Đưa ra dữ liệu thử nghiệm
predictions = random_forest_poly.predict_proba(poly_features_test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline_engineered.csv', index = False)

This scores 0.679 when submitted 

# Giải thích mô hình: Tầm quan trọng của Feature

In [ ]:
def plot_feature_importances(df):
    #Sắp xếp các tính năng theo mức độ quan trọng
    df = df.sort_values('importance', ascending = False).reset_index()
    
    #Bình thường quá các tính năng quan trọng
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Biểu đồ thể hiện tính năng quan trọng
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Đảo ngược chỉ số vẽ đồ thị
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
# Show biểu đồ
feature_importances_sorted = plot_feature_importances(feature_importances)

Như mong đợi, các tính năng quan trọng nhất là những tính năng liên quan đến EXT_SOURCE và DAYS_BIRTH.

# Light Gradient Boosting Machine

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    # Trích xuất id
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Trích dẫn nhãn
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Căn chỉnh dataframe theo các cột
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # Danh sách lưu trữ các chỉ số phân loại
        cat_indices = []
        
        # Lặp qua từng cột
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Ánh xạ các tính năng phân loại
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Ghi lại các chỉ số phân loại
                cat_indices.append(i)
    
    # Bắt lỗi nếu sơ đồ mã hóa nhãn không hợp lệ
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Trích xuất tên tính năng
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Danh sách ghi lại xác nhận và điểm train
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Thêm điểm tổng thể vào số liệu
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe của điểm xác nhận
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
submission, fi, metrics = model(app_train, app_test)
print('Baseline metrics')
print(metrics)

In [ ]:
fi_sorted = plot_feature_importances(fi)

In [ ]:
submission.to_csv('baseline_lgb.csv', index = False)

In [ ]:
app_train_domain['TARGET'] = train_labels

# Test the domain knolwedge features
submission_domain, fi_domain, metrics_domain = model(app_train_domain, app_test_domain)
print('Baseline with domain knowledge features metrics')
print(metrics_domain)

In [ ]:
fi_sorted = plot_feature_importances(fi_domain)

In [ ]:
submission_domain.to_csv('baseline_lgb_domain_features.csv', index = False)